In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re

%matplotlib inline

### Summary 
* 크롤링 페이지: http://land.naver.com
* URL 검색 인자: 
  http://land.naver.com/article/articleList.nhn?rletTypeCd=A01&tradeTypeCd=all&hscpTypeCd=A01:A03:A04&cortarNo=1168010600

* rletTypeCd: A01=아파트, A02=오피스텔, B01=분양권, 주택=C03, 토지=E03, 원룸=C01, 상가=D02, 사무실=D01, 공장=E02, 재개발=F01, 건물=D03
* tradeTypeCd (거래종류): all=전체, A1=매매, B1=전세, B2=월세, B3=단기임대
* hscpTypeCd (매물종류): 아파트=A01, 주상복합=A03, 재건축=A04 (복수 선택 가능)
* cortarNo(법정동코드): (예: 1168010600 서울시, 강남구, 대치동)

  ※ 법정동 코드에 대한 상세내용은 https://goo.gl/P6ni8Q 참조

In [9]:
url = 'https://land.naver.com/article/articleList.nhn?rletTypeCd=A01&tradeTypeCd=A1&hscpTypeCd=A01%3AA03%3AA04&cortarNo=1168010600'
result = requests.get(url)
bs_object= BeautifulSoup(result.content, 'html.parser')

table=bs_object.find('table')
trs = table.tbody.find_all('tr')

In [10]:
for tr in trs[::2]:
    tds = tr.find_all('td')
    cols = [' '.join(td.text.strip().split()) for td in tds]
    
    if '_thumb_image' not in tds[3]['class']: # 현장확인 날짜와 이미지가 없는 행
        cols.insert(3, '')

    거래 = cols[0]
    종류 = cols[1]
    확인일자 = datetime.strptime(cols[2], '%y.%m.%d.')
    현장확인 = cols[3]
    매물명 = cols[4]
    면적 = cols[5]
    공급면적 = re.findall('공급면적(.*?)㎡', 면적)[0].replace(',', '')
    전용면적 = re.findall('전용면적(.*?)㎡', 면적)[0].replace(',', '')
    공급면적 = float(공급면적)
    전용면적 = float(전용면적)
    층 = cols[6]
    매물가 = int(cols[7].replace(',', '')) 
    연락처 = cols[8]
        
    print(거래, 종류, 확인일자, 현장확인, 매물명, 공급면적, 전용면적, 층, 매물가, 연락처)

매매 재건축 2019-04-05 00:00:00 5/11 현장확인 은마 네이버부동산에서 보기 115.54 84.43 13/14 178000 대치탑공인중개사사무소 02-556-8200
매매 재건축 2019-04-05 00:00:00 5/15 현장확인 은마 네이버부동산에서 보기 101.52 76.79 7/14 163000 복음공인중개사사무소 02-556-8000
매매 재건축 2019-04-05 00:00:00 5/11 현장확인 은마 네이버부동산에서 보기 101.52 76.79 12/14 156000 중앙공인중개사사무소 02-566-6699
매매 재건축 2019-04-05 00:00:00 5/11 현장확인 은마 네이버부동산에서 보기 115.54 84.43 13/14 178000 명성공인중개사사무소 02-558-5333
매매 재건축 2019-04-03 00:00:00 5/12 현장확인 은마 네이버부동산에서 보기 101.52 76.79 1/14 153000 금마로공인중개사사무소 02-563-3600
매매 재건축 2019-04-02 00:00:00 5/11 현장확인 은마 네이버부동산에서 보기 115.54 84.43 13/14 178000 동아공인중개사사무소 02-565-4200
매매 아파트 2019-04-08 00:00:00  집주인 래미안대치팰리스1단지 네이버부동산에서 보기 88.15 59.99 9/18 197000 미래공인중개사사무소 02-556-6633
매매 재건축 2019-03-29 00:00:00 5/12 현장확인 은마 네이버부동산에서 보기 115.54 84.43 7/14 175000 대치탑공인중개사사무소 02-556-8200
매매 재건축 2019-03-29 00:00:00 5/15 현장확인 은마 네이버부동산에서 보기 101.52 76.79 7/14 155000 동아공인중개사사무소 02-565-4200
매매 재건축 2019-04-04 00:00:00 5/13 현장확인 은마 네이버부동산에서 보기 101.52 76.79 중/14 158000 성창공인중개사사무소 02

In [11]:

def get_naver_realasset(area_code, page=1):
    url = 'http://land.naver.com/article/articleList.nhn?' \
        + 'rletTypeCd=A01&tradeTypeCd=A1&hscpTypeCd=A01%3AA03%3AA04' \
        + '&cortarNo=' + area_code \
        + '&page=' + str(page)

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')

    table = soup.find('table')
    trs = table.tbody.find_all('tr')
    if '등록된 매물이 없습니다' in trs[0].text:
        return pd.DataFrame()
    
    value_list = []

    # 거래, 종류, 확인일자, 매물명, 면적(㎡), 층, 매물가(만원), 연락처
    for tr in trs[::2]:
        tds = tr.find_all('td')
        cols = [' '.join(td.text.strip().split()) for td in tds]

        if '_thumb_image' not in tds[3]['class']: # 현장확인 날짜와 이미지가 없는 행
            cols.insert(3, '')

        # print(cols)
        거래 = cols[0]
        종류 = cols[1]
        확인일자 = datetime.strptime(cols[2], '%y.%m.%d.')
        현장확인 = cols[3]
        매물명 = cols[4]
        면적 = cols[5]
        공급면적 = re.findall('공급면적(.*?)㎡', 면적)[0].replace(',', '')
        전용면적 = re.findall('전용면적(.*?)㎡', 면적)[0].replace(',', '')
        공급면적 = float(공급면적)
        전용면적 = float(전용면적)
        층 = cols[6]
        if cols[7].find('호가일뿐 실거래가로확인된 금액이 아닙니다') >= 0:
            pass # 단순호가 별도 처리하고자 하면 내용 추가
        매물가 = int(cols[7].split(' ')[0].replace(',', '')) 
        연락처 = cols[8]
        
        value_list.append([거래, 종류, 확인일자, 현장확인, 매물명, 공급면적, 전용면적, 층, 매물가, 연락처])
        
    cols = ['거래', '종류', '확인일자', '현장확인', '매물명', '공급면적', '전용면적', '층', '매물가', '연락처']
    df = pd.DataFrame(value_list, columns=cols)
    return df

In [12]:
df = get_naver_realasset('1168010600', 60)
df

,거래,종류,확인일자,현장확인,매물명,공급면적,전용면적,층,매물가,연락처
0,매매,아파트,2019-03-12,,대치삼성1차 네이버부동산에서 보기,124.57,97.35,22/24,185000,대치삼성공인 02-3288-2300
1,매매,아파트,2019-03-09,,집주인 래미안대치팰리스1단지 네이버부동산에서 보기,111.95,84.98,중/35,260000,태성부동산중개 02-566-7123
2,매매,주상복합,2019-03-19,,동민맥스빌(A동) 네이버부동산에서 보기,109.26,84.96,고/6,110000,세종공인중개사사무소02-558-1400
3,매매,아파트,2019-03-19,,하이캐슬 네이버부동산에서 보기,104.16,84.27,저/11,125000,세종공인중개사사무소02-558-1400
4,매매,아파트,2019-03-18,,래미안대치팰리스1단지 네이버부동산에서 보기,125.32,94.50,중/22,275000,청실래미안공인중개사사무소 02-557-5454
5,매매,아파트,2019-03-18,,래미안대치팰리스2단지 네이버부동산에서 보기,122.74,91.93,저/35,263000,정 공인중개사사무소 02-561-9558
6,매매,주상복합,2019-03-18,,대치우정에쉐르2 네이버부동산에서 보기,72.18,59.95,중/11,75000,(주)삼성종합부동산중개법인02-552-4028
7,매매,아파트,2019-03-18,,대치SK뷰 네이버부동산에서 보기,110.89,84.38,중/20,223000,정 공인중개사사무소 02-561-9558
8,매매,아파트,2019-03-16,,하이캐슬 네이버부동산에서 보기,104.16,84.27,중/11,125000,대치99공인중개사사무소 02-554-5999
9,매매,아파트,2019-03-11,,"한보미도맨션1,2차 네이버부동산에서 보기",152.72,128.01,5/14,238000,미도현대공인중개사사무소 02-552-4985
